In [1]:
import sys
import h5py
import ROOT

import numpy as np
import pandas as pd

from tqdm import tqdm

sys.path.append('..')
import utils_HDF5 as utils

delphes_path = '/usr/local/Delphes-3.4.2'
ROOT.gROOT.ProcessLine(f'.include {delphes_path}/')
ROOT.gROOT.ProcessLine(f'.include {delphes_path}/external/')
ROOT.gInterpreter.Declare(f'#include "{delphes_path}/classes/DelphesClasses.h"')
ROOT.gSystem.Load(f'{delphes_path}/install/lib/libDelphes')

MAX_JETS = 15

Welcome to JupyROOT 6.20/08


In [2]:
def DeltaR(eta1, phi1, eta2, phi2):
    dEta = eta1 - eta2
    dPhi = np.abs(phi1 - phi2)

    dPhi = np.where(dPhi > np.pi, 2 * np.pi - dPhi, dPhi)

    dR = np.sqrt(dPhi**2 + dEta**2)
    return dR

In [3]:
def print_cutflow_table(counts):
    n_tot = counts['total']
    n_previous = counts['total']
    print(f"{'Cut':<26} {'n':8} {'efficiency':8} {'passing rate':8}")
    for key, value in counts.items():
        if key not in ['3 Higgs', '2 Higgs', '1 Higgs']:
            print(f'{key:<26} {value:>8} {value / n_previous:>8.2f} {value / n_tot:>8.2f}')
            n_previous = value
        else:
            print(f'{key:<26} {value:>8} {value / n_previous:>8.2f} {value / n_tot:>8.2f}')

In [4]:
def print_matching_table(total_event, matched_event, nj=(6, 8), jet_type='Nj'):

    start, end = nj

    event_type = [f'{jet_type}={nj}' for nj in range(start, end)]
    event_type.append(f'{jet_type}>={end}')
    event_type.append('Total')

    event_fraction = [total_event[nj] / total_event.sum() for nj in range(start, end)]
    event_fraction.append(total_event[end:].sum() / total_event.sum())
    event_fraction.append(total_event.sum() / total_event.sum())

    matching_rate = [matched_event[nj] / total_event[nj].sum() for nj in range(start, end)]
    matching_rate.append(matched_event[end:].sum() / total_event[end:].sum())
    matching_rate.append(matched_event.sum() / total_event.sum())

        
    result = {
              'Event type': event_type,
              'Event Fraction': event_fraction,
              'Matching Rate': matching_rate,
             }

    df = pd.DataFrame(result)
    print(df)

    df_style = df.style.format({
            'Event Fraction': '{:.3f}',
            'Matching Rate': '{:.3f}',
        })
    print(df_style.to_latex(column_format='c|cc'))


In [5]:
def select_event(root_path, nbj_min, start, end):
    counts = {
        'total': 0,
        '>= 6 jets': 0,
        '>= 4 jets with pT > 40 GeV': 0,
        '>= 4 b-jets': 0,
        '3 Higgs': 0,
        '2 Higgs': 0,
        '1 Higgs': 0,
    }

    total_event_bjet = np.zeros(MAX_JETS + 1)
    matched_event_bjet = np.zeros(MAX_JETS + 1)
    total_event_Njet = np.zeros(MAX_JETS + 1)
    matched_event_Njet = np.zeros(MAX_JETS + 1)

    f = ROOT.TFile(root_path)
    tree = f.Get("Delphes")

    for i in tqdm(range(start, end)):
        tree.GetEntry(i)

        # 夸克資料
        # b夸克 衰變前的編號
        quarks_id = []
        quarks_Eta = []
        quarks_Phi = []
        quarks_Jet = np.array([-1, -1, -1, -1, -1, -1])

        # 找出 3 個 final Higgs
        final_h_index = []
        for index, particle in enumerate(tree.Particle):
            if particle.PID == 25:
                h = index
                d1 = tree.Particle[h].D1
                while tree.Particle[d1].PID == 25:
                    h = d1
                    d1 = tree.Particle[h].D1
                final_h_index.append(h)

        final_h_index = list(set(final_h_index))

        # 找出 6 個 final b quark
        for h in final_h_index:
            # h > b b~
            b1 = tree.Particle[h].D1
            b2 = tree.Particle[h].D2

            # 找出 b 衰變前的編號
            d1 = tree.Particle[b1].D1
            while abs(tree.Particle[d1].PID) == 5:
                b1 = d1
                d1 = tree.Particle[b1].D1

            # 找出 b~ 衰變前的編號
            d2 = tree.Particle[b2].D1
            while abs(tree.Particle[d2].PID) == 5:
                b2 = d2
                d2 = tree.Particle[b2].D1

            quarks_id.extend([b1, b2])

        quarks_Eta.extend(tree.Particle[quark].Eta for quark in quarks_id)
        quarks_Phi.extend(tree.Particle[quark].Phi for quark in quarks_id)

        # 事件中的 jet 資料
        jet_PT = np.array([jet.PT for jet in tree.Jet])
        jet_Eta = np.array([jet.Eta for jet in tree.Jet])
        jet_Phi = np.array([jet.Phi for jet in tree.Jet])
        jet_Mass = np.array([jet.Mass for jet in tree.Jet])
        jet_BTag = np.array([jet.BTag for jet in tree.Jet])

        # Jet 資料
        # |eta| < 2.5 & PT > 20 GeV
        eta_pt_cut = np.array((np.abs(jet_Eta) < 2.5) & (jet_PT > 20))
        # |eta| < 2.5 & PT > 40 GeV
        eta_pt40_cut = np.array((np.abs(jet_Eta) < 2.5) & (jet_PT > 40))

        nj = eta_pt_cut.sum()

        counts['total'] += 1
        # 至少要 6 jet
        if nj < 6:
            continue
        counts['>= 6 jets'] += 1
        # 至少要 4 jet pT > 40 GeV
        if eta_pt40_cut.sum() < 4:
            continue
        counts['>= 4 jets with pT > 40 GeV'] += 1
        nbj = np.array(jet_BTag[eta_pt_cut][:MAX_JETS]).sum()
        # 在前 MAX_JETS jets 中，至少要 nbj_min 個 b-jet
        if nbj < nbj_min:
            continue
        counts['>= 4 b-jets'] += 1
        total_event_bjet[nbj] += 1
        total_event_Njet[nj] += 1

        # PT = np.array(jet_PT[eta_pt_cut])
        Eta = np.array(jet_Eta[eta_pt_cut])
        Phi = np.array(jet_Phi[eta_pt_cut])
        # Mass = np.array(jet_Mass[eta_pt_cut])
        # BTag = np.array(jet_BTag[eta_pt_cut])

        # 找出每個夸克配對的 jet
        # more_than_1_jet = False
        for quark in range(len(quarks_Jet)):
            dR = DeltaR(quarks_Eta[quark], quarks_Phi[quark], Eta, Phi)
            if dR.min() < 0.4:
                quarks_Jet[quark] = np.argmin(dR)


        quark_jet = quarks_Jet.reshape(1, 6)

        h1_mask = utils.get_particle_mask(quark_jet, [0, 1])
        h2_mask = utils.get_particle_mask(quark_jet, [2, 3])
        h3_mask = utils.get_particle_mask(quark_jet, [4, 5])

        nh = [h1_mask, h2_mask, h3_mask].count(True)

        if nh == 3:
            counts['3 Higgs'] += 1
            matched_event_bjet[nbj] += 1
            matched_event_Njet[nj] += 1
        elif nh == 2:
            counts['2 Higgs'] += 1
        elif nh == 1:
            counts['1 Higgs'] += 1
       
    print_cutflow_table(counts)
    print_matching_table(total_event_bjet, matched_event_bjet, (4, 6), 'Nbj')
    print_matching_table(total_event_Njet, matched_event_Njet, (6, 8), 'Nj')

In [6]:
def compute_cutflow_table(root_path, nbj_min=0):
    f = ROOT.TFile(root_path)
    nevent = f.Get("Delphes").GetEntries()
    print(f'Number of events: {nevent}')

    start = 0
    end = nevent
    # end = 1000

    select_event(root_path, nbj_min, start, end)


In [7]:
root_path = './MG5/pythia_decay.root'
nbj_min = 4

compute_cutflow_table(root_path, nbj_min=nbj_min)

Number of events: 10000


100%|██████████| 10000/10000 [01:03<00:00, 156.44it/s]


Cut                        n        efficiency passing rate
total                         10000     1.00     1.00
>= 6 jets                      5209     0.52     0.52
>= 4 jets with pT > 40 GeV     4051     0.78     0.41
>= 4 b-jets                    2035     0.50     0.20
3 Higgs                         705     0.35     0.07
2 Higgs                         644     0.32     0.06
1 Higgs                         575     0.28     0.06
  Event type  Event Fraction  Matching Rate
0      Nbj=4        0.589189       0.293578
1      Nbj=5        0.325799       0.380090
2     Nbj>=6        0.085012       0.583815
3      Total        1.000000       0.346437
\begin{tabular}{c|cc}
 & Event type & Event Fraction & Matching Rate \\
0 & Nbj=4 & 0.589 & 0.294 \\
1 & Nbj=5 & 0.326 & 0.380 \\
2 & Nbj>=6 & 0.085 & 0.584 \\
3 & Total & 1.000 & 0.346 \\
\end{tabular}

  Event type  Event Fraction  Matching Rate
0       Nj=6         0.44914       0.251641
1       Nj=7         0.34398       0.388571
2     

Warning in <TStreamerInfo::BuildCheck>: 
   The StreamerInfo of class Track read from file ./MG5/pythia_decay.root
   has the same version (=3) as the active class but a different checksum.
   You should update the version to ClassDef(Track,4).
   Do not try to write objects with the current class definition,
   the files will not be readable.

Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 3 of class 'Track' is missing from 
the on-file layout version 3:
   float C; //
Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 3 of class 'Track' is missing from 
the on-file layout version 3:
   float Mass; //
Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 3 of class 'Track' is missing from 
the on-file layout version 3:
   float EtaOuter; //
Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout 

In [9]:
root_path = './MG5/gghhh_bsm/Events/run_01_decayed_1/tag_2_delphes_events.root'
nbj_min = 4

compute_cutflow_table(root_path, nbj_min=nbj_min)

Number of events: 100000


100%|██████████| 100000/100000 [10:06<00:00, 164.95it/s]


Cut                        n        efficiency passing rate
total                        100000     1.00     1.00
>= 6 jets                     61454     0.61     0.61
>= 4 jets with pT > 40 GeV    50341     0.82     0.50
>= 4 b-jets                   32337     0.64     0.32
3 Higgs                        9944     0.31     0.10
2 Higgs                       11341     0.35     0.11
1 Higgs                        8941     0.28     0.09
  Event type  Event Fraction  Matching Rate
0      Nbj=4        0.509107       0.186843
1      Nbj=5        0.368247       0.353796
2     Nbj>=6        0.122646       0.669440
3      Total        1.000000       0.307512
\begin{tabular}{c|cc}
 & Event type & Event Fraction & Matching Rate \\
0 & Nbj=4 & 0.509 & 0.187 \\
1 & Nbj=5 & 0.368 & 0.354 \\
2 & Nbj>=6 & 0.123 & 0.669 \\
3 & Total & 1.000 & 0.308 \\
\end{tabular}

  Event type  Event Fraction  Matching Rate
0       Nj=6        0.341126       0.218385
1       Nj=7        0.320469       0.321722
2     

Warning in <TStreamerInfo::BuildCheck>: 
   The StreamerInfo for version 2 of class GenParticle read from the file ./MG5/gghhh_bsm/Events/run_01_decayed_1/tag_2_delphes_events.root
   has a different checksum than the previously loaded StreamerInfo.
   Reading objects of type GenParticle from the file ./MG5/gghhh_bsm/Events/run_01_decayed_1/tag_2_delphes_events.root 
   (and potentially other files) might not work correctly.
   Most likely the version number of the class was not properly
   updated [See ClassDef(GenParticle,2)].
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float T; //number
vs
   float CtgTheta; //number
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float X; //number
vs
   float D0; //number
Warning in <TStreamerInfo